# Тестирование полносвязной спайковой сети на MNIST

In [10]:
import numpy as np
import theano
import theano.tensor as T
import lasagne
import matplotlib.pyplot as plt
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
import time
%matplotlib inline

In [2]:
%run ../SNN.ipynb

ERROR:root:Cell magic `%%cython` not found.


ImportError: No module named neuron

In [5]:
from mnist import load_dataset
X_train,y_train,X_val,y_val,X_test,y_test = load_dataset()

In [6]:
input_X = T.tensor4("X")
input_shape = [None,1,28,28]
target_y = T.vector("target Y integer",dtype='int32')

In [7]:
input_layer = lasagne.layers.InputLayer(shape = input_shape,input_var=input_X)

dense_1 = lasagne.layers.DenseLayer(input_layer, num_units=128, 
                                    nonlinearity=lasagne.nonlinearities.rectify,
                                   name='dense_1', b=None)

dense_2 = lasagne.layers.DenseLayer(dense_1, num_units=256, 
                                    nonlinearity=lasagne.nonlinearities.sigmoid,
                                   name='dense_2', b=None)

dense_3 = lasagne.layers.DenseLayer(dense_2, num_units=64, 
                                    nonlinearity=lasagne.nonlinearities.rectify,
                                   name='dense_2', b=None)

dense_output = lasagne.layers.DenseLayer(dense_3, num_units = 10,
                                        nonlinearity = lasagne.nonlinearities.softmax,
                                        name='output', b=None)

In [8]:
with np.load('dense_weights.npz') as f: 
    param_values = [f['arr_%d' % i]*10 for i in range(len(f.files))] 
    for i in param_values[-1]:
        for j in i:
            j = abs(j)
    for i in param_values:
        for j in i:
            for k, n in enumerate(j):
                if(n < 0.000001):
                    j[k] = 0.0

    
lasagne.layers.set_all_param_values(dense_output, param_values)

In [8]:
spiking_net = spiking_from_lasagne(dense_output, 0.2)

In [17]:
right_first = right_freq = right_betw = right_betw_min = 0.
leng = 10
n = 0
no_ans = 0
targ = y_train[:10]
answer = []
times = []
for num, x in enumerate(X_train[:10]):
    start_time = time.time()
    answer.append(spiking_from_lasagne(dense_output, 1.).get_output_for(x, 100))
    times.append(time.time() - start_time)
    print(targ[num], answer[-1])
for i, ans in enumerate(answer):
    first_spike = 400
    first_sp_ans = []
    for k, j in enumerate(ans):
        if(len(j) > 0):
            if(j[0] < first_spike):
                first_spike = j[0]
                first_sp_ans = [k]
            if(j[0] == first_spike):
                first_sp_ans.append(k)
    if(targ[i] in first_sp_ans):
        right_first += 1
    
    betw = [np.mean([j[l+1]-j[l] for l in np.arange(len(j)-1)]) for j in ans]
    betw_min_count = [np.min([j[l+1]-j[l] for l in np.arange(len(j)-1)]) if len([j[l+1]-j[l] for l in np.arange(len(j)-1)])>0 else 10000 for j in ans]
    betw_ans = []
    betw_min = 110
    betw_ans_min = []
    betw_min_min = 110
    
    ans = [len(j) for j in ans]
    freq_max = 0
    freq_ans = []
    for k, j in enumerate(ans):
        if (j > freq_max):
            freq_max = j
            freq_ans = [k]
        if (j == freq_max):
            freq_ans.append(k)
        if(betw[k] < betw_min):
            betw_min = j
            betw_ans = [k]
        if(betw[k] == betw_min):
            betw_ans.append(k)
        if(betw_min_count[k] < betw_min_min):
            betw_min_min = j
            betw_ans_min = [k]
        if(betw_min_count[k] == betw_min_min):
            betw_ans_min.append(k)
    if(targ[i] in betw_ans):
        right_betw += 1
    if(freq_ans == -1):
        no_ans += 1
    if(targ[i] in freq_ans):
        right_freq += 1
    if(targ[i] in betw_ans_min):
        right_betw_min += 1
        
print(right_first, right_freq, right_betw, right_betw_min)
print(right_first/leng, right_freq/leng, right_betw/leng, right_betw_min/leng)
print(np.mean(times), no_ans)

(5, [[20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 49, 51, 53, 55, 57, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 97, 99], [21, 23, 25, 27, 30, 32, 34, 36, 38, 40, 43, 45, 47, 49, 51, 53, 55, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 97, 99], [20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 49, 51, 53, 55, 57, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 97, 99], [20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 49, 51, 53, 55, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 97, 99], [21, 23, 25, 27, 30, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 98], [20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 49, 51, 53, 55, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 97, 99], [20, 23, 25, 27, 30, 32, 34, 36, 38, 40, 42, 44, 46, 49, 51, 53, 55, 58, 60, 62, 64

In [16]:
right_first = right_freq = right_betw = right_betw_min = 0.
no_ans = 0
for i, ans in enumerate(answer):
    first_spike = n + 2*leng
    first_sp_ans = np.array()
    for k, j in enumerate(ans):
        if(len(j) > 0):
            if(j[0] < first_spike):
                first_spike = j[0]
                first_sp_ans = np.array([k])
            if(j[0] == first_spike):
                first_sp_ans.append(k)
    if(targ[i] in first_sp_ans):
        right_first += 1
    
    betw = [np.mean([j[i+1]-j[i] for i in np.arange(len(j)-1)]) for j in ans]
    betw_min_count = [np.min([j[l+1]-j[l] for l in np.arange(len(j)-1)]) if len([j[l+1]-j[l] for l in np.arange(len(j)-1)])>0 else 10000 for j in ans]
    betw_ans = np.array()
    betw_min = 110
    betw_ans_min = np.array()
    betw_min_min = 110
    
    ans = [len(j) for j in ans]
    freq_max = 0
    freq_ans = np.array()
    for k, j in enumerate(ans):
        if (j > freq_max):
            freq_max = j
            freq_ans = np.array([k])
        if (j == freq_max):
            freq_ans.append(k)
        if(betw[k] < betw_min):
            betw_min = j
            betw_ans = np.array([k])
        if(betw[k] == betw_min):
            betw_ans.append(k)
        if(betw_min_count[k] < betw_min_min):
            betw_min_min = j
            betw_ans_min = np.array(k)
        if(betw_min_count[k] == betw_min_min):
            betw_ans_min.append(k)
    if(targ[i] in btw_ans):
        right_betw += 1
    if(freq_ans == -1):
        no_ans += 1
    if(targ[i] in freq_ans):
        right_freq += 1
    if(targ[i] in betw_ans_min):
        right_betw_min += 1
        
print(right_first, right_freq, right_betw, right_betw_min)
print(right_first/leng, right_freq/leng, right_betw/leng, right_betw_min/leng)
print(np.mean(times), no_ans)

NameError: name 'array' is not defined

In [ ]:
right_first = right_freq = right_betw = right_betw_min = 0.
leng = 10
n = 0
no_ans = 0
targ = y_train[10:20]
answer = []
times = []
for num, x in enumerate(X_train[10:20]):
    start_time = time.time()
    answer.append(spiking_from_lasagne(dense_output, 1.).get_output_for(x, 100))
    times.append(time.time() - start_time)
    print(targ[num], answer[-1])
for i, ans in enumerate(answer):
    first_spike = n + 2*leng
    first_sp_ans = -1
    for k, j in enumerate(ans):
        if(len(j) > 0):
            if(j[0] < first_spike):
                first_spike = j[0]
                first_sp_ans = k
    if(first_sp_ans == targ[i]):
        right_first += 1
    
    betw = [np.mean([j[i+1]-j[i] for i in np.arange(len(j)-1)]) for j in ans]
    betw_min_count = [np.min([j[i+1]-j[i] for i in np.arange(len(j)-1)]) for j in ans]
    betw_ans = -1
    betw_min = 110
    betw_ans_min = -1
    betw_min_min = 110
    
    ans = [len(j) for j in ans]
    freq_max = 0
    freq_ans = -1
    for k, j in enumerate(ans):
        if (j > freq_max):
            freq_ans = k
        if(j < betw_min):
            betw_ans = k
        if(j < betw_min_min):
            betw_ans_min = k
    if(betw_ans == targ[i]):
        right_betw += 1
    if(freq_ans == -1):
        no_ans += 1
    if(freq_ans == targ[i]):
        right_freq += 1
    if(betw_ans_min == targ[i]):
        right_betw_min += 1
        
print(right_first, right_freq, right_betw, right_betw_min)
print(right_first/leng, right_freq/leng, right_betw/leng, right_betw_min/leng)
print(np.mean(times), no_ans)

In [ ]:
right_first = right_freq = 0.
leng = 10
n = 0
no_ans = 0
pool = ThreadPool(4)
targ = y_train[100:200]
answer = []
times = []
for num, x in enumerate(X_train[100:200]):
    start_time = time.time()
    answer.append(spiking_from_lasagne(dense_output, .3).get_output_for(x, 100))
    times.append(time.time() - start_time)
    print(targ[num], answer[-1])
for i, ans in enumerate(answer):
    first_spike = n + 2*leng
    first_sp_ans = -1
    for k, j in enumerate(ans):
        if(len(j) > 0):
            if(j[0] < first_spike):
                first_spike = j[0]
                first_sp_ans = k
    if(first_sp_ans == targ[i]):
        right_first += 1
    
    ans = [len(j) for j in ans]
    freq_max = 0
    freq_ans = -1
    for k, j in enumerate(ans):
        if (j > freq_max):
            freq_ans = k
    if(freq_ans == -1):
        no_ans += 1
    if(freq_ans == targ[i]):
        right_freq += 1
        
print(right_first, right_freq)
print(right_first/leng, right_freq/leng)
print(times.mean(), no_ans)

In [ ]:
right_first = right_freq = 0.
leng = 10
n = 0
no_ans = 0
pool = ThreadPool(4)
targ = y_train[100:200]
answer = []
times = []
for num, x in enumerate(X_train[100:200]):
    start_time = time.time()
    answer.append(spiking_from_lasagne(dense_output, .01).get_output_for(x, 100))
    times.append(time.time() - start_time)
    print(targ[num], answer[-1])
for i, ans in enumerate(answer):
    first_spike = n + 2*leng
    first_sp_ans = -1
    for k, j in enumerate(ans):
        if(len(j) > 0):
            if(j[0] < first_spike):
                first_spike = j[0]
                first_sp_ans = k
    if(first_sp_ans == targ[i]):
        right_first += 1
    
    ans = [len(j) for j in ans]
    freq_max = 0
    freq_ans = -1
    for k, j in enumerate(ans):
        if (j > freq_max):
            freq_ans = k
    if(freq_ans == -1):
        no_ans += 1
    if(freq_ans == targ[i]):
        right_freq += 1
        
print(right_first, right_freq)
print(right_first/leng, right_freq/leng)
print(times.mean(), no_ans)

In [ ]:
for i in np.arange(10):
    print(y_test[i])
    plt.imshow(X_test[i,0])
    plt.show()

In [ ]:
params = [spiking_net.layers[i].weights.tolist() for i in np.arange(1, len(spiking_net.layers))]
par = " ".join(map(str, ["\n".join(map(str, [" ".join(map(str, ["%.6f" % k for k in j])) for j in i])) for i in params]))
#for i in params:
    #for j in i:
        #par+=str(" ".join(map(str, ["%.6f" % k for k in j])))+'\n'
        len(spiking_net.layers)
        i.neur_size for i in spiking_net.layers
    #par+='\n'

In [ ]:
info = {'threshold':0.5, 'tau_m':4, 'tau_s':2, 'tau_r':20, 'layersNumber':444,
        'sizes':[4, 5, 6], 'synapsesPerConnection':1, 'spikesPerSynapse':2, 'exitTime':1000,\
        'weights':''}


In [ ]:
f = open('params.txt', 'w')
f.writelines([i+': '+str(info[i])+'\n' for i in info])

In [ ]:
for i in param_values:
    for j in i:
        f.writelines(str(" ".join(map(str, ["%.6f" % k for k in j])))+'\n')
f.close()

In [ ]:
params = np.array([spiking_net.layers[i].weights.tolist() for i in np.arange(1, len(spiking_net.layers))])

In [ ]:
len(params[0][1])

In [ ]:
btw str(" ".join(map(str, params)))

In [ ]:
def write_to_file(net, filename='params.txt'):
    params = [net.layers[i].weights.tolist() for i in np.arange(1, len(net.layers))]
    info = {'threshold':0.5, 'tau_m':4, 'tau_s':2, 'tau_r':20, 'layersNumber':len(net.layers),
        'sizes':[i.neur_size for i in net.layers], 'synapsesPerConnection':1, 'spikesPerSynapse':2, 'exitTime':1000,\
        'weights':''}
    f = open(filename, 'w')
    f.writelines([i+': '+str(info[i])+'\n' for i in info])
    for i in params:
        for j in i:
            f.writelines(str(" ".join(map(str, ["%.6f" % k for k in j])))+'\n')
    f.close()

In [ ]:
a = [[], [2, 4]]
np.min(a, axis=1)